# Routing

You can find shortest path between two nodes in the graph, using a modal subgraph or otherwise.

In [1]:
from genet import read_matsim
import os

path_to_matsim_network = '../example_data/pt2matsim_network'

network = os.path.join(path_to_matsim_network, 'network.xml')
schedule = os.path.join(path_to_matsim_network, 'schedule.xml')
vehicles = os.path.join(path_to_matsim_network, 'vehicles.xml')
n = read_matsim(
    path_to_network=network, 
    epsg='epsg:27700', 
    path_to_schedule=schedule, 
    path_to_vehicles=vehicles
)
# you don't need to read the vehicles file, but doing so ensures all vehicles
# in the schedule are of the expected type and the definition of the vehicle
# is preserved
n.print()

Graph info: Name: Network graph
Type: MultiDiGraph
Number of nodes: 1662
Number of edges: 3166
Average in degree:   1.9049
Average out degree:   1.9049 
Schedule info: Schedule:
Number of services: 9
Number of routes: 68
Number of stops: 118


In [2]:
from_node = '101982'
to_node = '101991'

The following will use the whole graph disregarding modes.

In [3]:
n.find_shortest_path(from_node, to_node)

['2030', '2453', '296', '3147', '3287']

The following will compute a modal subgraph and use it for routing. You can also pass a list e.g. `['bike', 'walk']`. 

In [4]:
n.find_shortest_path(from_node, to_node, modes='car')

['2030', '2453', '296', '3147', '3287']

If you have many node pairs to process, it may be beneficial to compute the modal subgraph of interest first and pass that to the method.

In [5]:
car_g = n.modal_subgraph('car')
n.find_shortest_path(from_node, to_node, subgraph=car_g)

['2030', '2453', '296', '3147', '3287']

Specifying `'modes'` on top of giving the `subgraph` will also use given modes for preferential treatment if there is
ambiguity in which link should be chosen for the route (remember, there can be several links between the same two 
nodes). For example, if using mode `'bus'` and there are two links to choose from, one with modes: `['car', 'bus']` and
the other with just `['bus']`, preference will be given to the link dedicated to that mode. Otherwise, preference
will be given to links with higher `freespeed`.

In [6]:
n.find_shortest_path(from_node, to_node, subgraph=car_g, modes=['bus'])

['2030', '2453', '296', '3147', '3287']

The anser is the same because this network does not have multiple bus only links.

You can also choose to return the chain of nodes instead:

In [7]:
n.find_shortest_path(from_node, to_node, return_nodes=True)

['101982', '1611082208', '2489433555', '25495406', '1514726033', '101991']

If you are looking to route a PT service in the `Network`'s `Schedule` you have a few methods to choose from.

If the `Network` graph or the data on your graph has changed and you want to reroute a service that was previously snapped to the netowrk and routed you can use the `reroute` method. You can pass either a service or route ID, they should be unique and will be handled appropriately. You can specify `additional_modes`, e.g. `car` mode that will allow buses to use the links that allow the car mode as well as the route's own mode `bus`. Following this, the mode on the used links will be updated with the mode `bus`.

In [8]:
n.reroute('VJ375a660d47a2aa570aa20a8568012da8497ffecf', additional_modes={'car'})

2021-10-25 11:01:18,058 - Checking `linkRefId`s of the Route: `VJ375a660d47a2aa570aa20a8568012da8497ffecf` are present in the graph
2021-10-25 11:01:18,062 - Rerouting Route `VJ375a660d47a2aa570aa20a8568012da8497ffecf`
2021-10-25 11:01:18,275 - Changed Route attributes for 1 routes


It could happen that the snapping of a route or service is no longer valid after the changes, or you want to add something completely new. In this case you can use the `route_service` method which will find new links to snap and route the service on the network. You will need to have a solver set up, e.g. [CBC](https://projects.coin-or.org/Cbc). Again, you can specify `additional_modes` as in the method above. There are other parameters you can make use of, more details can be found in 5.2 which talks about modifying Schedules.

In [9]:
n.route_service('20274', additional_modes={'car'})

2021-10-25 11:01:22,931 - Routing Service 20274 with modes = {'bus'}
2021-10-25 11:01:23,089 - Building Maximum Stable Set for PT graph with 8 stops and 6 edges
2021-10-25 11:01:28,067 - Passing problem to solver
2021-10-25 11:01:28,069 - Initializing ordered Set vertices with a fundamentally unordered data source (type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo
2021-10-25 11:01:28,088 - Passing problem to solver
2021-10-25 11:01:28,645 - Stop ID changes detected for Routes: {'VJ6c64ab7b477e201cae950efde5bd0cb4e2e8888e', 'VJ375a660d47a2aa570aa20a8568012da8497ffecf', 'VJ812fad65e7fa418645b57b446f00cba573f2cdaf'}
2021-10-25 11:01:28,651 - Changed Route attributes for 3 routes
2021-10-25 11:01:28,672 - Changed Link attributes for 41 links


If you are creating a `Network` and want to snap and route the entire `Schedule`, or a larger number of services, the method `route_schedule` is advised. Bear in mind though that it will struggle with large networks and big and complicated schedules. Similar parameters apply to this method as the one abov

In [10]:
unsnapped_service_ids = n.route_schedule(services=['20274', '15660'])

2021-10-25 11:01:28,682 - Building Spatial Tree
2021-10-25 11:01:35,162 - Extracting Modal SubTree for modes: {'bus'}
2021-10-25 11:01:35,271 - Routing Service 20274 with modes = {'bus'}
2021-10-25 11:01:35,272 - Building Maximum Stable Set for PT graph with 8 stops and 6 edges
2021-10-25 11:01:39,047 - Passing problem to solver
2021-10-25 11:01:39,049 - Initializing ordered Set vertices with a fundamentally unordered data source (type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo
2021-10-25 11:01:39,053 - Passing problem to solver
2021-10-25 11:01:39,470 - Routing Service 15660 with modes = {'bus'}
2021-10-25 11:01:39,473 - Building Maximum Stable Set for PT graph with 5 stops and 3 edges
2021-10-25 11:01:44,381 - This Maximum Stable Set Problem is partially viable.
2021-10-25 11:01:44,382 - Maximum Stable Set problem to snap the PT graph to the network is partially viable, meaning not all stops have found a link to snap to within the distance_threshold.Part

Some services may fail to snap. Method above returns IDs of the services which failed. It is worth re-running these, with the same or different parameters. Failing that, a service can also be teleported using the following method. If the stops are already snapped (i.e. have a `linkRefId`), those links will still be used as references, unless the link is no longer in the network.

In [11]:
n.teleport_service(service_ids='17732')

2021-10-25 11:01:45,514 - Added 0 nodes
2021-10-25 11:01:46,333 - Generated 0 link ids.
2021-10-25 11:01:46,350 - Added 8 links
2021-10-25 11:01:46,356 - Changed Stop attributes for 10 stops
2021-10-25 11:01:46,361 - Changed Route attributes for 2 routes


In [12]:
n.schedule.stop('490004695A.link:3017').print()

Stop ID: 490004695A.link:3017
Projection: epsg:27700
Lat, Lon: 51.51433903, -0.12977799
linkRefId: 3017


In [13]:
list(n.schedule['17732'].routes())[0].ordered_stops

['490004695A.link:3017',
 '490000235C.link:3068',
 '490000089A.link:823',
 '490000252X.link:86',
 '490000078Q.link:1239']

In [14]:
list(n.schedule['17732'].routes())[0].route

['3017',
 'artificial_link===from:21665081===to:5434424322',
 '3068',
 'artificial_link===from:3519133221===to:108045',
 '823',
 'artificial_link===from:3079462268===to:4543005956',
 '86',
 'artificial_link===from:25714232===to:4543005959',
 '1239']